In [1]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
import cv2, numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import applications
from PIL import Image
import xml.etree.ElementTree
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter

Using TensorFlow backend.


In [37]:
df = pd.DataFrame(columns=['FileName', 'Species', 'ClassId'])
count= 0
for root, dirs, files in os.walk('Data'):
    for name in files:
        if name.endswith(".xml"):
            root = root.replace("\\", "/")
            xmlFile = xml.etree.ElementTree.parse(root + '/' + name).getroot()
            FileName = root + '/' + xmlFile[0].text
            Species = xmlFile[1].text
            LearnTag = xmlFile[9].text
            if LearnTag != 'Train':
                print(LearnTag)
            ClassId = int(xmlFile[10].text)
            df.loc[count] = [FileName, Species, ClassId]
            count += 1

In [93]:
X_train, X_test, y_train, y_test = train_test_split(df, df['ClassId'],
                                                    stratify=df['ClassId'],
                                                    random_state=123,
                                                    test_size=0.3)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, X_train['ClassId'],
                                                    stratify=X_train['ClassId'],
                                                    random_state=123,
                                                    test_size=0.2)
#print(Counter(X_train['ClassId']))
#print(Counter(X_test['ClassId']))
print(X_train.shape)

(122, 3)


In [104]:
def centering_image(img):
    size = [256,256]
    
    img_size = img.shape[:2]
    
    # centering
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized


def get_images(dataFrame):
    x = []
    for i, file_path in enumerate(dataFrame.FileName):
        #read image
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #resize
        if(img.shape[0] > img.shape[1]):
            tile_size = (int(img.shape[1]*256/img.shape[0]),256)
        else:
            tile_size = (256, int(img.shape[0]*256/img.shape[1]))
        
        #centering
        img = centering_image(cv2.resize(img, dsize=tile_size))

        #out put 224*224px 
        img = img[16:240, 16:240]
        x.append(img)
    return x

x_train = get_images(X_train)
y_train = np.array(y_train)
x_train = np.array(x_train)

x_validation = get_images(X_validation)
y_validation = np.array(y_validation)
x_validation = np.array(x_validation)

x_test = get_images(X_test)
y_test = np.array(y_test)
x_test = np.array(x_test)

In [105]:
print('x_train', x_train.shape)
print('y_train', y_train.shape)
print('x_validation', x_validation.shape)
print('y_validation', y_validation.shape)
print('x_test', x_test.shape)
print('y_test', y_test.shape)

x_train (122, 224, 224, 3)
y_train (122,)
x_validation (31, 224, 224, 3)
y_validation (31,)
x_test (66, 224, 224, 3)
y_test (66,)


In [111]:
nbClasses = 3

img_rows, img_cols, img_channel = 224, 224, 3
base_model = applications.xception.Xception(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 10 000 classes
predictions = Dense(nbClasses, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional Xception layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(x.layers):
    print(i, layer.name)

AttributeError: 'Tensor' object has no attribute 'layers'

In [110]:
batch_size = 32
epochs = 50

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

print(x_train.shape)
print(x_validation.shape)
model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_validation, y_validation),
    callbacks=[ModelCheckpoint('Xception-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

(122, 224, 224, 3)
(31, 224, 224, 3)


ValueError: Error when checking target: expected dense_6 to have shape (3,) but got array with shape (1,)

In [ ]:
# we chose to train the top 2 Xception blocks, i.e. we will freeze
# the first 115 layers and unfreeze the rest:
for layer in model.layers[:115]:
    layer.trainable = False
for layer in model.layers[115:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_validation, y_validation),
    callbacks=[ModelCheckpoint('Xception-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('putois.jpg')  # this is a PIL image
img = img.resize([224,224], Image.ANTIALIAS)
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely
        
print(x.shape)

In [3]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)
    
    return model

In [10]:
if __name__ == "__main__":
    im = cv2.resize(cv2.imread('putois.jpg'), (224, 224)).astype(np.float32)
    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.transpose((1,0,2))
    im = np.expand_dims(im, axis=0)

    # Test pretrained model
    model = VGG_16('vgg16_weights_tf_dim_ordering_tf_kernels.h5')
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')
    out = model.predict(im)
    print(np.argmax(out))

C:\Users\axel3\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
C:\Users\axel3\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
C:\Users\axel3\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\axel3\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  if sys.path[0] == '':
C:\Users\axel3\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  app.launch_new_instance()
C:\Users\a

380
